# fuzzyCmeans Implementation from scratch on cardiac spectf imaging dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import random
import operator
import math


from collections import defaultdict


from sklearn import datasets
from sklearn.datasets import make_blobs

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

import os, cv2, json


np.random.seed(42)

> # load the dataset

In [2]:
WORK_DIR='../input/spectf/'
os.listdir(WORK_DIR)

['SPECTF_test.csv']

In [3]:
test=pd.read_csv(os.path.join(WORK_DIR,"SPECTF_test.csv"))
test

,Attr_1,Attr_2,Attr_3,Attr_4,Attr_5,Attr_6,Attr_7,Attr_8,Attr_9,Attr_10,...,Attr_36,Attr_37,Attr_38,Attr_39,Attr_40,Attr_41,Attr_42,Attr_43,Attr_44,Class
0,32,41,76,34,65,53,30,54,16,51,...,43,67,61,29,31,17,8,18,11,Yes
1,76,65,60,40,32,34,65,50,53,37,...,51,65,59,43,39,35,30,24,21,Yes
2,60,51,75,60,65,45,64,55,55,61,...,68,69,63,62,53,57,31,46,30,Yes
3,64,60,71,69,71,65,66,64,68,59,...,55,66,58,65,65,75,64,61,56,Yes
4,65,69,66,76,58,67,65,72,66,64,...,64,71,70,72,69,70,65,63,61,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,70,75,72,72,67,71,71,78,63,67,...,67,68,70,70,71,64,67,56,54,No
106,59,57,67,71,66,68,68,70,56,62,...,64,56,53,71,68,64,63,56,56,No
107,67,64,73,75,77,77,74,70,65,62,...,64,65,60,68,75,74,80,67,68,No
108,68,65,72,72,47,74,76,74,67,66,...,69,71,73,73,75,68,56,58,44,No


In [4]:
target_values = test['Class'].copy()
test.drop('Class', axis=1, inplace=True, errors="ignore")


In [5]:
target_values

0      Yes
1      Yes
2      Yes
3      Yes
4      Yes
      ... 
105     No
106     No
107     No
108     No
109     No
Name: Class, Length: 110, dtype: object

In [6]:
test

,Attr_1,Attr_2,Attr_3,Attr_4,Attr_5,Attr_6,Attr_7,Attr_8,Attr_9,Attr_10,...,Attr_35,Attr_36,Attr_37,Attr_38,Attr_39,Attr_40,Attr_41,Attr_42,Attr_43,Attr_44
0,32,41,76,34,65,53,30,54,16,51,...,31,43,67,61,29,31,17,8,18,11
1,76,65,60,40,32,34,65,50,53,37,...,62,51,65,59,43,39,35,30,24,21
2,60,51,75,60,65,45,64,55,55,61,...,76,68,69,63,62,53,57,31,46,30
3,64,60,71,69,71,65,66,64,68,59,...,66,55,66,58,65,65,75,64,61,56
4,65,69,66,76,58,67,65,72,66,64,...,71,64,71,70,72,69,70,65,63,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,70,75,72,72,67,71,71,78,63,67,...,66,67,68,70,70,71,64,67,56,54
106,59,57,67,71,66,68,68,70,56,62,...,62,64,56,53,71,68,64,63,56,56
107,67,64,73,75,77,77,74,70,65,62,...,61,64,65,60,68,75,74,80,67,68
108,68,65,72,72,47,74,76,74,67,66,...,64,69,71,73,73,75,68,56,58,44


In [7]:
X=test
y=target_values

In [8]:
clusters=len(np.unique(y))

In [9]:
X=X.values

In [10]:
X

array([[32, 41, 76, ...,  8, 18, 11],
       [76, 65, 60, ..., 30, 24, 21],
       [60, 51, 75, ..., 31, 46, 30],
       ...,
       [67, 64, 73, ..., 80, 67, 68],
       [68, 65, 72, ..., 56, 58, 44],
       [66, 54, 69, ..., 68, 59, 59]])

In [11]:
class_yes=[]
class_no=[]

data=X.tolist()
y
for i in range(len(y)):
    if(y[i]=='Yes'):
        class_yes.append(data[i])
    elif(y[i]=='No'):
        class_no.append(data[i])

In [12]:
X


array([[32, 41, 76, ...,  8, 18, 11],
       [76, 65, 60, ..., 30, 24, 21],
       [60, 51, 75, ..., 31, 46, 30],
       ...,
       [67, 64, 73, ..., 80, 67, 68],
       [68, 65, 72, ..., 56, 58, 44],
       [66, 54, 69, ..., 68, 59, 59]])

In [13]:
y

0      Yes
1      Yes
2      Yes
3      Yes
4      Yes
      ... 
105     No
106     No
107     No
108     No
109     No
Name: Class, Length: 110, dtype: object

In [14]:
class_yes

[[32,
  41,
  76,
  34,
  65,
  53,
  30,
  54,
  16,
  51,
  61,
  43,
  74,
  70,
  62,
  21,
  34,
  42,
  61,
  37,
  58,
  66,
  54,
  49,
  17,
  19,
  11,
  42,
  53,
  30,
  71,
  9,
  61,
  16,
  31,
  43,
  67,
  61,
  29,
  31,
  17,
  8,
  18,
  11],
 [76,
  65,
  60,
  40,
  32,
  34,
  65,
  50,
  53,
  37,
  66,
  53,
  33,
  31,
  30,
  30,
  69,
  75,
  64,
  45,
  68,
  65,
  57,
  43,
  23,
  23,
  53,
  77,
  48,
  32,
  55,
  32,
  48,
  23,
  62,
  51,
  65,
  59,
  43,
  39,
  35,
  30,
  24,
  21],
 [60,
  51,
  75,
  60,
  65,
  45,
  64,
  55,
  55,
  61,
  66,
  74,
  61,
  50,
  62,
  41,
  70,
  63,
  60,
  62,
  76,
  69,
  70,
  54,
  51,
  47,
  77,
  80,
  69,
  48,
  74,
  59,
  72,
  57,
  76,
  68,
  69,
  63,
  62,
  53,
  57,
  31,
  46,
  30],
 [64,
  60,
  71,
  69,
  71,
  65,
  66,
  64,
  68,
  59,
  63,
  67,
  68,
  64,
  73,
  59,
  73,
  60,
  72,
  56,
  77,
  68,
  69,
  69,
  66,
  59,
  57,
  40,
  55,
  53,
  69,
  63,
  71,
  63,
  6

# Function to calculate euclidean distance

In [15]:
def euclidean_distance(point1,point2):
    dis=0
    for i in range(len(point1)):
        dis+=(point1[i]-point2[i])**2
    return dis**0.5

# fuzzyCMeans Implementation

In [16]:
# utility values
m=2
n=len(test)
c=2
p=len(data[0])
max_iter=100


In [17]:
def initialize_membership_matrix(n,c):
    member_mat=list()
    for i in range(n):
        random_list=[random.random() for x in range(c)]
        summation=sum(random_list)
        for i in range(len(random_list)):
            random_list[i]=random_list[i]/summation
        member_mat.append(random_list)
    return member_mat

def update_centroid(member_mat):
    centroids={}

    for j in range(c):
        temp=[]
        for k in range(p):
            
            add=0
            for i in range(n):
                add+=member_mat[i][j]**m
            x=0
            for i in range(n):
                x+=(member_mat[i][j]**m)*(data[i][k])
            val=x/add
            temp.append(val)
        centroids[j]=temp
    return centroids

def update_membership_matrix(member_mat,centroids):
    ratio=float(2/(m-1))

    for i in range(n):
        distances=list()
        for j in range(c):
            distances.append(euclidean_distance(data[i],centroids[j]))
        for j in range(c):
            den = sum([math.pow(float(distances[j]/distances[q]), ratio) for q in range(c)])
            member_mat[i][j] = float(1/den) 
           
            
    return member_mat
        
                
    
def find_cluster(member_mat):
    clusters=list()
    for i in range(n):
        max_val, idx = max((val, idx) for (idx, val) in enumerate(member_mat[i]))
        clusters.append(idx)
    return clusters

def check(old_member_mat,member_mat):
    diff=0
    for i in range(n):
        for j in range(c):
            diff+=old_member_mat[i][j]-member_mat[i][j]
    if(diff<0.01):
        return True
    return False

def fuzzy_c_mean():
    
    member_mat=initialize_membership_matrix(n,c)


    for i in range(max_iter):
        centroids=update_centroid(member_mat)
        old_member_mat=member_mat
        member_mat=update_membership_matrix(member_mat,centroids)
        cluster=find_cluster(member_mat)
   

    return cluster,centroids




        
          

# prediction

In [18]:
clusters,centroids=fuzzy_c_mean()
print("Final Centroid points are:")
print(centroids)


Final Centroid points are:
{0: [67.44811989662884, 66.85293831701826, 69.48595141584852, 70.22447457813092, 67.1727148973485, 67.19047824349623, 71.32507494056152, 71.21061919990693, 63.89618939500051, 63.51075471481833, 70.08059358976229, 70.68686260178968, 67.97659751978232, 68.9788822751547, 64.16790097579059, 61.911785035529824, 68.41305073968623, 67.36276209107649, 66.54371721561404, 64.38219703920514, 74.26268618627526, 72.94569707773607, 68.34936313353329, 68.08386600515446, 60.19552622852439, 59.25050435959892, 62.82011603645129, 61.65925625929214, 64.27093710891722, 63.359503481806826, 71.67468670610558, 72.17304518646115, 62.14653925613799, 62.404138072489395, 65.3912301079379, 64.82110535035721, 65.7244482630481, 64.90140922701019, 68.90305301720116, 67.50753774005265, 66.06412743951361, 64.29626544194326, 55.569726503085526, 54.13139612400519], 1: [67.44283108764728, 66.85616269377664, 69.49585118073084, 70.24820351763988, 67.19087761510313, 67.22760185256676, 71.3294893865

# confusion matrix

In [19]:
def confusion_matrix(clusters,y):
    c_matrix=[[0 for i in range(2)] for j in range(2)]
    tp = [0, 0]
    tn = [0, 0]
    fp = [0, 0]
    fn = [0, 0]
    for i in range(len(y)):
        if clusters[i] == 1 and y[i] == 'Yes':
            tp[0] = tp[0] + 1
        if clusters[i] == 0 and y[i] == 'No':
            tn[0] = tn[0] + 1
        if clusters[i] == 1 and y[i] == 'No':
            fp[0] = fp[0] + 1
        if clusters[i] == 0 and y[i] == 'Yes':
            fn[0] = fn[0] + 1
        
    for i in range(len(y)):
        
        if clusters[i] == 0 and y[i] == 'Yes':
            tp[1] = tp[1] + 1
        if clusters[i] == 1 and y[i] == 'No':
            tn[1] = tn[1] + 1
        if clusters[i] == 0 and y[i] == 'No':
            fp[1] = fp[1] + 1
        if clusters[i] == 1 and y[i] == 'Yes':
            fn[1] = fn[1] + 1
      
    x=max((tp[0]+tn[0]),(tp[1]+tn[1]))
    if x==tp[0]+tn[0]:
            c_matrix[0][0]=tp[0]
            c_matrix[1][1]=tn[0]
            c_matrix[0][1]=fn[0]
            c_matrix[1][0]=fp[0]
            return c_matrix
    else:
            c_matrix[0][0]=tp[1]
            c_matrix[1][1]=tn[1]
            c_matrix[0][1]=fn[1]
            c_matrix[1][0]=fp[1]
            return c_matrix


# performance metrics
            
class Metrics:
    
    def __init__(self,confusion_m):
        self.confusion_m=confusion_m
        self.total=np.sum(confusion_m)
        self.diagonal=np.sum(np.diag(confusion_m))
    
    def accuracy(self):
        accuracy=(self.diagonal/self.total)
        return accuracy
    
    def recall(self):
        recall=np.diag(self.confusion_m)/np.sum(self.confusion_m,axis=1)
        recall=np.mean(recall)
        return recall
    
    def precision(self):
        precision=np.diag(self.confusion_m)/np.sum(self.confusion_m,axis=0)
        precision=np.mean(precision)
        return precision
    
    def f1_score(self,precision,recall):
        f1_score=(2*precision*recall)/(precision+recall)
        
        return f1_score


# performance metrics

In [20]:
matrix=confusion_matrix(clusters,y)
performance=Metrics(matrix)

accuracy=performance.accuracy()
recall=performance.recall()
precision=performance.precision()
f1_score=performance.f1_score(precision,recall)

print('confusion matrix is:',end='\n')
print(np.array(matrix),end='\n')

print("Accuracy of the model is {}".format(accuracy*100))
print("Recall of the model is {}".format(recall*100))
print("Precision of the model is {}".format(precision*100))
print("F1-Score of the model is {}".format(f1_score*100))

confusion matrix is:
[[31 24]
 [ 4 51]]
Accuracy of the model is 74.54545454545455
Recall of the model is 74.54545454545453
Precision of the model is 78.28571428571429
F1-Score of the model is 76.3698164513936
